# Renaming Notebook for Drone images

Images are taken and renamed to the following format

DJIP4_TH_TKB_05_20181222T1743Z_0001.JPG

{dronetype}\_{country}\_{surveycode}\_{surveynumber}\_{localtime}\_{image_counter}.JPG


### 1. Run Exiftool on the input directory

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj 
import os
import shutil
import PIL
import xarray as xr
from functools import partial
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import yaml
import glob

In [90]:
sourcepath ='/home/mor582/seagrass/Thailand/'
outputdir = '/home/mor582/seagrass'

In [97]:
def get_longitude(item):
    longitude =float(item[0]) + float(item[2][0:-1])/60 + float(item[3][0:-1])/3600
    return (longitude)

def save_exif(survey,inputdir,outputdir,copyfiles=True):
    print(inputdir)
    jsonfile = os.path.join(inputdir,'input.json')
    if not os.path.exists(jsonfile):
        !exiftool -r -ext JPG -a -json $inputdir > $jsonfile
    xifdata =pd.read_json(jsonfile)
    xifdata['OriginalImageNumber'] =pd.to_numeric(xifdata.FileName.str.extract('.*_(?P<ImageNumber>\d*)')['ImageNumber'])
    xifdata.index = pd.to_datetime(xifdata.DateTimeOriginal,format='%Y:%m:%d %H:%M:%S')
    xifdata.index.name='Time'
    xifdata.DateTimeOriginal =pd.to_datetime(xifdata.DateTimeOriginal,format='%Y:%m:%d %H:%M:%S')
    xifdata.sort_index(inplace=True)
    xifdata['Hem']=np.nan
    xifdata.loc[xifdata['GPSLatitudeRef']=='North','Hem']=1
    xifdata.loc[xifdata['GPSLatitudeRef']=='South','Hem']=-1
    # Drop all pictures with No GPS
    xifdata = xifdata[(~xifdata['GPSLongitude'].isna() | ~xifdata['GPSLatitude'].isna())]
    xifdata['Longitude'] = np.nan
    xifdata['Latitude'] = np.nan
    xifdata['longitude']=xifdata['GPSLongitude'].str.split(' ',expand=True).apply(get_longitude,axis=1)
    xifdata['latitude']=xifdata['GPSLatitude'].str.split(' ',expand=True).apply(get_longitude,axis=1) * xifdata['Hem']
    xifdata['LocalTime']=xifdata.index
    xifdata['Interval']=pd.to_datetime(xifdata.LocalTime).diff().dt.total_seconds()
    xifdata['Leg'] =0
    xifdata.loc[xifdata['Interval']>8,'Leg'] =1
    xifdata['Leg'] = xifdata['Leg'].cumsum()
    xifdata['LocalTime'] =  xifdata.index
    xifdata['LocalTime'] = xifdata['LocalTime'].dt.strftime('%Y%m%dT%H%M%S')+timedif
    xifdata['Counter'] = 1
    xifdata['Counter'] = xifdata['Counter'].cumsum()
    xifdata['NewName']=xifdata.apply(lambda item: f"{survey['dronetype']}_{survey['camera']}_{survey['country']}_{survey['surveycode']}_{survey['surveynumber']:03}_{item.LocalTime}_{item.Counter:04}.JPG", axis=1)    
    xifdata = xifdata[['AbsoluteAltitude','CamReverse','CameraPitch','CameraRoll','CameraYaw',
                         'CircleOfConfusion','DateTimeOriginal','ExifImageHeight','ExifImageWidth',
                         'ExposureTime','FNumber','FOV','FileSource','FlightPitchDegree','FlightRollDegree',
                         'FlightYawDegree','FocalLength','FocalLength35efl','FocalLengthIn35mmFormat',
                         'GPSAltitude','GPSAltitudeRef','GPSLatitude','GPSLatitudeRef','GPSLongitude',
                         'GPSLongitudeRef','GPSPosition','GPSVersionID','GainControl',
                         'GimbalPitchDegree','GimbalReverse','GimbalRollDegree','GimbalYawDegree',
                         'ISO','ImageHeight','ImageWidth','Orientation','Pitch','RelativeAltitude',
                         'ResolutionUnit','Roll','Saturation','ScaleFactor35efl','SerialNumber',
                         'Sharpness','ShutterSpeed','ShutterSpeedValue','Software',
                         'SourceFile','SpeedX','SpeedY','SpeedZ','SubjectDistance',
                         'SubjectDistanceRange','Version','WhiteBalance','Yaw', 'Hem', 'Longitude', 'Latitude',
                         'LocalTime', 'Counter', 'NewName','FileName','OriginalImageNumber','Leg']]
    
    xifdata['SurveyName'] =survey['longname']
    outputpath =os.path.join(outputdir,country,f"{survey['surveycode']}_{survey['surveynumber']:03}")
    os.makedirs(outputpath,exist_ok=True)
    xifdata.to_csv(f"{outputpath}/{survey['dronetype']}_{survey['country']}_{survey['surveycode']}_{survey['surveynumber']:03}_RENAME.csv",index=True)
    if copyfiles:
        for index,item in xifdata.iterrows():
            if not os.path.exists(os.path.join(outputpath,item['NewName'])):
                print(os.path.join(outputpath,item['NewName']))
                shutil.copy(item['SourceFile'],os.path.join(outputpath,item['NewName']))
    return xifdata



In [99]:
processfiles = glob.glob(sourcepath+'**/process.yml',recursive=True)
for pfile in processfiles:
    with open(pfile) as file:
        survey = yaml.load(file,Loader=yaml.FullLoader)
        for s in survey['survey']:
            print(s)
            save_exif(survey['survey'][s],os.path.split(processfiles[0])[0],outputdir)
            
            

KMK02_001
/home/mor582/seagrass/Thailand/KohMook/MK_Right
KMK01_001
/home/mor582/seagrass/Thailand/KohMook/MK_Right
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164550+07_0001.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164553+07_0002.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164556+07_0003.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164558+07_0004.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164601+07_0005.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164603+07_0006.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164606+07_0007.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164608+07_0008.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164609+07_0009.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T164636+07_0010.JPG
/home/mor582/seagrass/TH

/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165048+07_0095.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165051+07_0096.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165053+07_0097.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165056+07_0098.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165058+07_0099.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165101+07_0100.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165103+07_0101.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165106+07_0102.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165109+07_0103.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165110+07_0104.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T165120+07_0105.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_

/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T170433+07_0191.JPG
/home/mor582/seagrass/TH/KMK01_001/DJIP4_RGB_TH_KMK01_001_20200721T170436+07_0192.JPG
